# 1. 기본 KeyBERT

In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [2]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

In [3]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/split_by_doc.pkl')

In [4]:
df2 = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/SBERT_data.pkl')

In [5]:
def find_content(data, split_data, title, subtitle):

  # 1) desc -> desc 출력
  if subtitle == 'desc':
    return data[title]['desc']

  # 2) 그 외 특정 목차
  # 해당 목차의 내용을 출력
  else:
    try:
      j = data[title]['subtitles']['name'].index(subtitle)
      st = data[title]['subtitles']['idx'][j] + 1
      try:
        ed = data[title]['subtitles']['idx'][j+1]
      except:
        ed = -1
      output = ' '.join(split_data[title].split('\n')[st:ed])
      if len(output)>2000:
        return '글자수 초과'
      else:
        return output
    except:
      return '잘못된 입력입니다.'

In [6]:
import re

def remove_(text):
    ''' 불필요한 기호 제거 '''
    text = re.sub("'", " ", text)
    text = re.sub('"', " ", text)
    text = re.sub('《', " ", text)
    text = re.sub('》', " ", text)
    text = re.sub('<', " ", text)
    text = re.sub('>', " ", text) 
    text = re.sub('〈', " ", text)
    text = re.sub('〉', " ", text)   
    text = re.sub("\(", " ", text)
    text = re.sub("\)", " ", text)
    text = re.sub("‘", " ", text)
    text = re.sub("’", " ", text)      
    text = re.sub("/", "", text)      
    return text

In [35]:
title = input('title : ')
subtitle = input('subtitle : ')
find_content(df2, df, title, subtitle)

title : 지미 카터
subtitle : 생애


''

In [19]:
title = [df.keys()]

In [20]:
title_li = title[0][:1000]

In [21]:
len(title_li)

1000

In [9]:
title_li_1 = []

for i in range(len(title_li)):
  title_li_1.append(remove_(title_li[i]))

title_li = title_li_1

In [10]:
title_li[:10]

['지미 카터', '수학', '수학 상수', '문학', '나라 목록', '화학', '체첸 공화국', '맥스웰 방정식', '초월수', '음계']

In [196]:
okt = Okt()

doc = input('질문 : ')
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

질문 : 물
품사 태깅 10개만 출력 : [('물', 'Noun')]
명사 추출 : 물


In [ ]:
n_gram_range = (2, 3)

if len(tokenized_nouns) == 1:
  qas_candidates = np.array([tokenized_nouns], dtype = object)
  qas_embedding = model.encode(qas_candidates)
else:
  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  qas_candidates = count.get_feature_names_out()
  qas_embedding = model.encode(qas_candidates)

print('trigram 개수 :', qas_candidates)
print('trigram 다섯개만 출력 :',qas_embedding)

In [69]:
qas_candidates[:1]

array(['지미 카터'], dtype=object)

In [98]:
np.array([title_li[961]], dtype = object)

array(['케냐'], dtype=object)

In [167]:
# count = CountVectorizer(ngram_range=n_gram_range).fit([title_li[960]])
candidates = count.get_feature_names_out()

In [158]:
n_gram_range = (2, 3)
title_embedding_li = []

for i in tqdm(range(len(title_li))):
  if len(title_li[i]) == 1:
    candidates = np.array([title_li[i]], dtype = object)
    title_embedding = model.encode(candidates)
  else:
    count = CountVectorizer(ngram_range=n_gram_range).fit([title_li[i]])
    candidates = count.get_feature_names_out()
    title_embedding = model.encode(candidates)
  title_embedding_li.append(title_embedding)

len(title_embedding_li)

  0%|          | 0/1000 [00:00<?, ?it/s]

1000

In [ ]:
title_embedding_li[:10]

In [180]:
candidates

array(['수학 대해'], dtype=object)

In [181]:
doc_embedding = model.encode(doc)
qas_embedding = model.encode(qas_candidates)

In [199]:
distances_li = []
keywords_li = []

for i in tqdm(range(len(title_li))):
  distances = cosine_similarity(qas_embedding, title_embedding_li[i])
  distances_li.append(distances)
  keywords_li.append(max(map(max, distances_li[i])))
  title_idx = keywords_li.index(max(keywords_li))

print(title_li[title_idx])

  0%|          | 0/1000 [00:00<?, ?it/s]

물


In [157]:
print(title_li[title_idx])

에드워드 텔러


In [150]:
print(distances_li[2])
print(max(map(max, distances_li[2])))

[[0.54543614 0.2696863  0.28626525]
 [0.7431009  0.3335848  0.35350275]
 [0.7227668  0.29650843 0.29806393]]
0.7431009


# SBERT 모델

In [11]:
# model 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [40]:
# 질문

okt = Okt()

doc = input('질문 : ')
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

질문 : 지미 카터가 뭐야?
품사 태깅 10개만 출력 : [('지미', 'Noun'), ('카터', 'Noun'), ('가', 'Josa'), ('뭐', 'Noun'), ('야', 'Josa'), ('?', 'Punctuation')]
명사 추출 : 지미 카터 뭐


In [41]:
# 질문 embedding

n_gram_range = (2, 3)

try:
  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  qas_candidates = count.get_feature_names_out()
  qas_embedding = model.encode(qas_candidates)
except:
  qas_candidates = np.array([tokenized_nouns], dtype = object)
  qas_embedding = model.encode(qas_candidates)

print('trigram 개수 :', qas_candidates)
# print('trigram 다섯개만 출력 :',qas_embedding)

trigram 개수 : ['지미 카터']


In [22]:
# title embedding list 만들기

n_gram_range = (2, 3)
title_embedding_li = []

for i in tqdm(range(len(title_li))):
  try:
    count = CountVectorizer(ngram_range=n_gram_range).fit([title_li[i]])
    candidates = count.get_feature_names_out()
    title_embedding = model.encode(candidates)
  except:
    candidates = np.array([title_li[i]], dtype = object)
    title_embedding = model.encode(candidates)
  title_embedding_li.append(title_embedding)

len(title_embedding_li)

  0%|          | 0/1000 [00:00<?, ?it/s]

1000

In [42]:
distances_li = []
keywords_li = []

for i in tqdm(range(len(title_li))):
  distances = cosine_similarity(qas_embedding, title_embedding_li[i])
  distances_li.append(distances)
  keywords_li.append(max(map(max, distances_li[i])))
  title_idx = keywords_li.index(max(keywords_li))

print(title_li[title_idx])

  0%|          | 0/1000 [00:00<?, ?it/s]

지미 카터


In [224]:
title = title_li[title_idx]
subtitle = input('subtitle : ')
find_content(df2, df, title, subtitle)

subtitle : 공식


'잘못된 입력입니다.'